In [1]:
import pandas as pd
import numpy as np

In [2]:
from freq_pd import proc_freq
from missing_columns_pd import miss_cols

In [3]:
dict_dtypes ={'Invoice/Item Number':str,
 'Date':str,
 'Store Number':str,
 'Store Name':str,
 'Address':str,
 'City':str,
 'Zip Code':str,
 'Store Location':str,
 'County Number':str,
 'County':str,
 'Category':str,
 'Category Name':str,
 'Vendor Number':str,
 'Vendor Name':str,
 'Item Number':str,
 'Item Description':str,
 'Pack':int,
 'Bottle Volume (ml)':int,
 'State Bottle Cost':float,
 'State Bottle Retail':float,
 'Bottles Sold':int,
 'Sale (Dollars)':float,
 'Volume Sold (Liters)':float,
 'Volume Sold (Gallons)':float}

In [4]:
liquor_2021 = pd.read_csv(r'C:\Users\dgarb\OneDrive\Desktop\Capstone Data\liquor_2021\liquor_2021.csv', dtype=dict_dtypes)

In [5]:
liquor_2021.shape

(2622712, 24)

In [6]:
miss_cols(liquor_2021)

[('Store Location', 311710, 0.1189),
 ('Address', 142, 0.0001),
 ('City', 142, 0.0001),
 ('Zip Code', 142, 0.0001),
 ('County Number', 142, 0.0001),
 ('County', 142, 0.0001),
 ('Vendor Number', 4, 0.0),
 ('Vendor Name', 4, 0.0)]

In [7]:
# liquor_2022 = 

In [8]:
pd.set_option('display.max_columns', None)
liquor_2021.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,INV-41517100002,11/01/2021,5417,Casey's General Store # 2792/Cedar Rapids,9001 6th Street SW,Cedar Rapids,52404,NaN,57,LINN,1031100,American Vodkas,434,LUXCO INC,36308,Hawkeye Vodka,6,1750,7.17,10.76,6,64.56,10.50,2.77
1,INV-41518800037,11/01/2021,4921,Market Express,5340 16th Ave SW,Cedar Rapids,52404,NaN,57,LINN,1011200,Straight Bourbon Whiskies,065,Jim Beam Brands,19061,Jim Beam Mini,12,50,7.00,10.50,1,10.50,0.05,0.01
2,INV-41518900014,11/01/2021,5687,Casey's General Store #3756 / Cedar Rapids,4560 16th Ave SW,Cedar Rapids,52404,NaN,57,LINN,1012100,Canadian Whiskies,260,DIAGEO AMERICAS,11296,Crown Royal,12,750,16.59,24.89,12,298.68,9.00,2.37
3,INV-41521200068,11/01/2021,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.774728 41.560871),77,POLK,1081500,Triple Sec,434,LUXCO INC,86250,Juarez Triple Sec,12,750,2.67,4.01,12,48.12,9.00,2.37
4,INV-41523800012,11/01/2021,4379,Kum & Go #2091 / Ashworth / WDM,5969 Ashworth Rd,West Des Moines,50266,NaN,77,POLK,1051100,American Brandies,205,E & J Gallo Winery,52596,E & J VS,12,750,6.50,9.75,5,48.75,3.75,0.99


In [9]:
liquor_2021['trans_date'] = pd.to_datetime(liquor_2021['Date'])

create transactions DataFrame

In [10]:
transactions = liquor_2021[['Vendor Number', 'Store Number','Item Number', 'Date', 'Bottle Volume (ml)', 'Volume Sold (Liters)', 'Bottles Sold','Sale (Dollars)','State Bottle Cost','State Bottle Retail','Category','trans_date']]

In [11]:
df_prod1 = liquor_2021.sort_values(by=['trans_date']).drop_duplicates(['Item Number'],keep='last')

In [12]:
products = df_prod1[['Item Number', 'Item Description',  'Category']]

create item_trans

In [13]:
item_trans = transactions.merge(products, how='inner', on= ['Category','Item Number'])

In [14]:
item_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2604721 entries, 0 to 2604720
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Vendor Number         object        
 1   Store Number          object        
 2   Item Number           object        
 3   Date                  object        
 4   Bottle Volume (ml)    int32         
 5   Volume Sold (Liters)  float64       
 6   Bottles Sold          int32         
 7   Sale (Dollars)        float64       
 8   State Bottle Cost     float64       
 9   State Bottle Retail   float64       
 10  Category              object        
 11  trans_date            datetime64[ns]
 12  Item Description      object        
dtypes: datetime64[ns](1), float64(4), int32(2), object(6)
memory usage: 258.3+ MB


create a dataframe to use for clustering products (item_info_to_cluster.csv)

In [15]:
df_bottles = item_trans.groupby(['Item Number','Item Description','Bottle Volume (ml)']).agg({'State Bottle Retail':'mean','Bottles Sold':'sum'}).reset_index()

In [62]:
dict_dtypes = {'Item_Number':str,
'Bottle_Volume':int,
'Bottles_Sold':int,
'Bottle_Retail':float}

In [63]:
df_product_clusters= pd.read_csv(r'C:\Users\dgarb\OneDrive\Desktop\Capstone Data\liquor_2021\product_clusters.csv',dtype=dict_dtypes, index_col=0)

In [16]:
# df_bottles #= df_bottles.set_index('Item Number') 750 ml is a wine bottle size

In [17]:
df_bottles.to_csv(r'C:\Users\dgarb\OneDrive\Desktop\Capstone Data\liquor_2021\item_info_to_cluster.csv')

In [56]:
df_counties = liquor_2021.groupby(['County']).agg({'Store Number' : 'nunique','Zip Code':'nunique' }).reset_index()

In [61]:
df_counties

,County,Store Number,Zip Code
0,ADAIR,6,4
1,ADAMS,2,1
2,ALLAMAKEE,10,4
3,APPANOOSE,9,2
4,AUDUBON,4,2
...,...,...,...
114,WOODBURY,48,12
115,WORTH,4,2
116,WRIGHT,12,5
117,Wapello,1,1


In [59]:
filt = df_counties['County']=='POLK'

In [60]:
df_counties[filt]

,County,Store Number,Zip Code
90,POLK,241,26


In [53]:
df_counties['Store Number'].max()

241

In [54]:
filt = df_counties['Store Number'] == 241

df_counties[filt]

,County,Store Number
90,POLK,241


In [55]:
liquor_2021.loc[liquor_2021['County']=='POLK',:]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),trans_date
3,INV-41521200068,11/01/2021,2648,Hy-Vee #4 / WDM,555 S 51st St,West Des Moines,50265,POINT (-93.774728 41.560871),77,POLK,1081500,Triple Sec,434,LUXCO INC,86250,Juarez Triple Sec,12,750,2.67,4.01,12,48.12,9.00,2.37,2021-11-01
4,INV-41523800012,11/01/2021,4379,Kum & Go #2091 / Ashworth / WDM,5969 Ashworth Rd,West Des Moines,50266,NaN,77,POLK,1051100,American Brandies,205,E & J Gallo Winery,52596,E & J VS,12,750,6.50,9.75,5,48.75,3.75,0.99,2021-11-01
18,INV-41556400064,11/02/2021,2535,Hy-Vee Food Store #1 / WDM,1700 Valley West Dr,West Des Moines,50265,NaN,77,POLK,1032100,Imported Vodkas,035,BACARDI USA INC,34433,Grey Goose,12,750,15.00,22.50,12,270.00,9.00,2.37,2021-11-02
32,INV-41598000062,11/03/2021,2502,Hy-Vee Wine and Spirits / Ankeny,410 North Ankeny Blvd,Ankeny,50021,NaN,77,POLK,1011200,Straight Bourbon Whiskies,259,Heaven Hill Brands,17958,Evan Williams Black,6,1750,16.00,24.00,30,720.00,52.50,13.86,2021-11-03
33,INV-41598000181,11/03/2021,2502,Hy-Vee Wine and Spirits / Ankeny,410 North Ankeny Blvd,Ankeny,50021,NaN,77,POLK,1062500,Flavored Rum,370,PERNOD RICARD USA,42712,Malibu Coconut Mini,12,50,4.95,7.43,3,22.29,0.15,0.03,2021-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622423,INV-41504800005,10/31/2021,4301,Sahota Food Mart,1805 SE 14th St,Des Moines,50320,POINT (-93.59684900000002 41.57222),77,POLK,1071100,Cocktails /RTD,395,PROXIMO,59161,1800 Ultimate Peach Margarita,6,1750,10.04,15.06,6,90.36,10.50,2.77,2021-10-31
2622439,INV-41504300007,10/31/2021,5098,Best Food Mart 3 LLC,1621 University Ave,Des Moines,50311,NaN,77,POLK,1041100,American Dry Gins,370,PERNOD RICARD USA,32231,Seagrams Extra Dry Gin Mini,12,50,3.30,4.95,12,59.40,0.60,0.15,2021-10-31
2622589,INV-41504300017,10/31/2021,5098,Best Food Mart 3 LLC,1621 University Ave,Des Moines,50311,NaN,77,POLK,1081600,Whiskey Liqueur,421,SAZERAC COMPANY INC,64864,Fireball Cinnamon Whiskey,24,375,5.33,8.00,24,192.00,9.00,2.37,2021-10-31
2622628,INV-41504400005,10/31/2021,5906,Polk City Liquor,201 N 3rd St STE A and B,Polk City,50226,NaN,77,POLK,1011300,Single Barrel Bourbon Whiskies,306,MISSISSIPPI RIVER DISTIL,27677,Cody Road Single Barrel,6,750,25.00,37.50,2,75.00,1.50,0.39,2021-10-31


create stores DataFrame

In [18]:
df_store1 = liquor_2021.sort_values(by=['trans_date']).drop_duplicates(['Store Number'],keep='last')

In [19]:
stores = df_store1[['Store Number', 'Store Name', 'Address', 'City', 'Zip Code', 'Store Location', 'County Number', 'County']]

In [20]:
df_vendor1 =  liquor_2021.sort_values(by=['trans_date']).drop_duplicates(['Vendor Number'],keep='last')

In [21]:
vendors = df_vendor1[['Vendor Number', 'Vendor Name']]

read in Iowa Liquor Products file

In [22]:
dytype_dict2 = {'Item Number':str, 'Vendor':str, 'Proof':int, 'Vendor Name':str, 'Age':int}

Iowa_Liquor_Products = pd.read_csv(r'C:\Users\dgarb\OneDrive\Desktop\Capstone Data\Iowa_Liquor_Products.csv', dtype=dytype_dict2 )

In [23]:
pd.set_option('display.max_columns', None)
Iowa_Liquor_Products.head(2)

,Item Number,Category Name,Item Description,Vendor,Vendor Name,Bottle Volume (ml),Pack,Inner Pack,Age,Proof,List Date,UPC,SCC,State Bottle Cost,State Case Cost,State Bottle Retail,Report Date
0,901141,Special Order Items,Libertine Absinthe(French Absinthe) Mini,885,YAHARA BAY DISTILLERS INC,50,6,1,0,114,04/20/2012,NaN,10892060104050,4.82,28.9,7.23,03/01/2022
1,65017,Imported Cordials & Liqueurs,Galliano Liqueur,305,Mhw LTD,375,12,1,0,84,10/01/2012,8.091511e+10,10892060104050,10.75,129.0,16.13,03/01/2022


In [24]:
product_proof = Iowa_Liquor_Products[['Item Number','Age','Proof']]

In [25]:
df_categ1 = liquor_2021.sort_values(by=['trans_date']).drop_duplicates(['Category'],keep='last')

In [26]:
categories = df_categ1[[ 'Category', 'Category Name']]

In [27]:
miss_cols(categories)

[]

In [28]:
dict_categ = dict(list(zip(categories['Category'],categories['Category Name'])))

dict_list = sorted(dict_categ.items(), key = lambda kv:(kv[1], kv[0]))

# dict_categ['1081300']='American Cordials & Liqueurs'

# ('1081300', 'American Cordials & Liqueur'),
#  ('1081000', 'American Cordials & Liqueurs'),
# ('1081400', 'American Schnapps')
# ('1081100', 'Coffee Liqueurs')
# ('1081200', 'Cream Liqueurs')

# ('1082100', 'Imported Cordials & Liqueur'),
#  ('1082000', 'Imported Cordials & Liqueurs'),
#     ('1082200', 'Imported Schnapps')
#     ('1081600', 'Whiskey Liqueur')
# dict_list

In [29]:
# proc_freq(categories,'Category Name')

In [30]:
imp_liquers_trans = transactions.loc[(transactions['Category']== '1082100') | (transactions['Category']=='1082000') | (transactions['Category']=='1082200')]

In [31]:
imp_liquers_trans.shape

(102138, 12)

In [32]:
imp_liquers_trans['Vendor Number'].nunique()

36

In [33]:
imp_liquers_trans['Store Number'].nunique()

1878

In [34]:
liquor_2021['County'].nunique()

119

In [35]:
imp_liquers_trans.merge(stores, how='inner', on='Store Number')['County'].nunique()

118

In [36]:
imp_liquers_trans['Item Number'].nunique()

169

Why doesn't this work?

In [37]:
filt = transactions['Category'].isin(['1081300','1081000','1081400','1081100','1081200'])

In [38]:
american_liquers_trans = transactions.loc[filt,:]
american_liquers_trans.shape

(217973, 12)

In [39]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2622712 entries, 0 to 2622711
Data columns (total 12 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Vendor Number         object        
 1   Store Number          object        
 2   Item Number           object        
 3   Date                  object        
 4   Bottle Volume (ml)    int32         
 5   Volume Sold (Liters)  float64       
 6   Bottles Sold          int32         
 7   Sale (Dollars)        float64       
 8   State Bottle Cost     float64       
 9   State Bottle Retail   float64       
 10  Category              object        
 11  trans_date            datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int32(2), object(5)
memory usage: 220.1+ MB
